| Etapa                      | Tecnologia recomendada                              | Justificativa                                           |
| -------------------------- | --------------------------------------------------- | ------------------------------------------------------- |
| Orquestração mensal        | `cron` no App Service ou Azure Functions            | Automação simples                                       |
| Ambiente isolado           | Docker                                              | Facilita dependências (`duckdb`, `pandas`, `azure-sdk`) |
| Extração + Upload Bronze   | Python puro (`requests`, `zipfile`, `DataLake SDK`) | Você já faz isso bem                                    |
| Transformações Silver/Gold | `Pandas` ou `DuckDB`                                | Leves, simples, ideais para CSV                         |
| Deploy                     | Azure App Service + GitHub Actions                  | CI/CD automatizado                                      |


cnes-data-pipeline/
│
├── Dockerfile
├── requirements.txt
├── app.py                  ← ponto de entrada principal
├── extract/
│   └── extract_cnes.py     ← download e bronze upload
├── transform/
│   ├── silver.py           ← limpeza e joins
│   └── gold.py             ← agregações e finais
├── utils/
│   └── azure_datalake.py   ← funções para leitura/escrita
└── cron/
    └── crontab.txt         ← para rodar mensalmente no container


In [ ]:
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta
from azure.storage.filedatalake import DataLakeServiceClient
import zipfile

# ==== CONFIGURAÇÕES ====
ano_mes = (datetime.today() - relativedelta(months=3)).strftime("%Y%m")
account_name = "caiostorageaccountdev"
account_key = "IExmwuvHY7HX2rYSouaQAhAEjFrXYMfGpKIKFFMAVlz8B5rOn8/wQX8Np7yXGQSgneFel74yDwq7+AStvLQjJQ=="
file_system_name = "bronze"  # nome do container
target_dir = f"cnes/{ano_mes}"

# ==== CAMINHOS ====
zip_path = f"/Users/caio.maximiano/pessoal/cnes-project-analysis/local_storage/zip/BASE_DE_DADOS_CNES_{ano_mes}.ZIP"
local_folder = f"/Users/caio.maximiano/pessoal/cnes-project-analysis/local_storage/csv/cnes_extract_{ano_mes}"
url = f"https://cnes.datasus.gov.br/EstatisticasServlet?path=BASE_DE_DADOS_CNES_{ano_mes}.ZIP"

# ==== DOWNLOAD DO ZIP ====
print(f"Iniciando download para o período {ano_mes}")
os.system(f"wget -O {zip_path} {url}")

# ==== EXTRAÇÃO DO ZIP ====
extract_path = local_folder  # mesmo destino esperado depois
os.makedirs(extract_path, exist_ok=True)

print(f"Extraindo arquivos para {extract_path}")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# ==== CONEXÃO COM O DATA LAKE ====
datalake_client = DataLakeServiceClient(
    account_url=f"https://{account_name}.dfs.core.windows.net",
    credential=account_key
)
file_system_client = datalake_client.get_file_system_client(file_system_name)

# ==== UPLOAD DE CSVs PARA O DATA LAKE ====
for root, _, files in os.walk(local_folder):
    for file_name in files:
        if file_name.lower().endswith(".csv"):
            local_path = os.path.join(root, file_name)
            blob_path = f"{target_dir}/{file_name}"

            print(f"Enviando {file_name} para abfss://{file_system_name}@{account_name}.dfs.core.windows.net/{blob_path}")

            file_client = file_system_client.get_file_client(blob_path)
            with open(local_path, "rb") as data:
                file_client.upload_data(
                    data,
                    overwrite=True,
                    max_concurrency=4,            # número de uploads paralelos
                    chunk_size=4 * 1024 * 1024    # 4 MB por chunk (ajustável)
                )


print("Upload finalizado com DataLakeServiceClient.")


Iniciando download para o período 202504


--2025-07-13 21:19:35--  https://cnes.datasus.gov.br/EstatisticasServlet?path=BASE_DE_DADOS_CNES_202504.ZIP
Resolving cnes.datasus.gov.br (cnes.datasus.gov.br)... 189.28.139.195
Connecting to cnes.datasus.gov.br (cnes.datasus.gov.br)|189.28.139.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/Users/caio.maximiano/pessoal/cnes-project-analysis/local_storage/zip/BASE_DE_DADOS_CNES_202504.ZIP’

     0K .......... .......... .......... .......... .......... 1.54M
    50K .......... .......... .......... .......... .......... 2.17M
   100K .......... .......... .......... .......... .......... 2.65M
   150K .......... .......... .......... .......... .......... 3.14M
   200K .......... .......... .......... .......... .......... 4.51M
   250K .......... .......... .......... .......... .......... 2.17M
   300K .......... .......... .......... .......... .......... 4.49M
   350K .......... .......... .......... .........

Upload finalizado com DataLakeServiceClient.


... .......... .......... ..........  376K
616650K .......... .......... .......... .......... .......... 5.39M
616700K .......... .......... .......... .......... .......... 1.56M
616750K .......... .......... .......... .......... .......... 3.85M
616800K .......... .......... .......... .......... .......... 3.61M
616850K .......... .......... .......... .........             1.90M=3m56s

2025-07-13 21:23:32 (2.55 MB/s) - ‘/Users/caio.maximiano/pessoal/cnes-project-analysis/local_storage/zip/BASE_DE_DADOS_CNES_202504.ZIP’ saved [631694960]



In [ ]:
# ==== UPLOAD DE CSVs PARA O DATA LAKE ====
for root, _, files in os.walk(local_folder):
    for file_name in files:
        if file_name.lower().endswith(".csv"):
            local_path = os.path.join(root, file_name)
            blob_path = f"{target_dir}/{file_name}"

            print(f"Enviando {file_name} para abfss://{file_system_name}@{account_name}.dfs.core.windows.net/{blob_path}")

            file_client = file_system_client.get_file_client(blob_path)
            with open(local_path, "rb") as data:
                file_client.upload_data(
    data,
    overwrite=True,
    max_concurrency=4,            # número de uploads paralelos
    chunk_size=4 * 1024 * 1024    # 4 MB por chunk (ajustável)
)

print("Upload finalizado com DataLakeServiceClient.")

In [ ]:
# ==== EXTRAÇÃO DO ZIP ====
extract_path = local_folder  # mesmo destino esperado depois
os.makedirs(extract_path, exist_ok=True)

print(f"Extraindo arquivos para {extract_path}")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# ==== CONEXÃO COM O DATA LAKE ====
datalake_client = DataLakeServiceClient(
    account_url=f"https://{account_name}.dfs.core.windows.net",
    credential=account_key
)
file_system_client = datalake_client.get_file_system_client(file_system_name)

# ==== UPLOAD DE CSVs PARA O DATA LAKE ====
for root, _, files in os.walk(local_folder):
    for file_name in files:
        if file_name.lower().endswith(".csv"):
            local_path = os.path.join(root, file_name)
            blob_path = f"{target_dir}/{file_name}"

            print(f"Enviando {file_name} para abfss://{file_system_name}@{account_name}.dfs.core.windows.net/{blob_path}")

            file_client = file_system_client.get_file_client(blob_path)
            with open(local_path, "rb") as data:
                file_client.upload_data(
                    data,
                    overwrite=True,
                    max_concurrency=4,            # número de uploads paralelos
                    chunk_size=4 * 1024 * 1024    # 4 MB por chunk (ajustável)
                )


print("Upload finalizado com DataLakeServiceClient.")


Extraindo arquivos para /Users/caio.maximiano/pessoal/cnes-project-analysis/local_storage/csv/cnes_extract_202504
Enviando rlTipoEstabAtividade202504.csv para abfss://bronze@caiostorageaccountdev.dfs.core.windows.net/cnes/202504/rlTipoEstabAtividade202504.csv
Enviando tbMotivoDesativacao202504.csv para abfss://bronze@caiostorageaccountdev.dfs.core.windows.net/cnes/202504/tbMotivoDesativacao202504.csv
Enviando tbTipoEqSubTipo202504.csv para abfss://bronze@caiostorageaccountdev.dfs.core.windows.net/cnes/202504/tbTipoEqSubTipo202504.csv
Enviando rlEstabProfComissao202504.csv para abfss://bronze@caiostorageaccountdev.dfs.core.windows.net/cnes/202504/rlEstabProfComissao202504.csv
Enviando tbAvaliacao202504.csv para abfss://bronze@caiostorageaccountdev.dfs.core.windows.net/cnes/202504/tbAvaliacao202504.csv
Enviando tbAtividade202504.csv para abfss://bronze@caiostorageaccountdev.dfs.core.windows.net/cnes/202504/tbAtividade202504.csv
Enviando rlEstabEqpUnidApoio202504.csv para abfss://bronze@c